In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [4]:
from google.colab import drive
drive.flush_and_unmount()

In [2]:

import cv2
import numpy as np
import random
import os

DATASET_DIR = "/content/drive/MyDrive/Dataset/ECG/"  # change this to your folder
OUTPUT_DIR = "/content/drive/MyDrive/Dataset/ECG/augmented"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Input folder:", DATASET_DIR)
print("Output folder:", OUTPUT_DIR)


Input folder: /content/drive/MyDrive/Dataset/ECG/
Output folder: /content/drive/MyDrive/Dataset/ECG/augmented


In [ ]:
files = os.listdir(DATASET_DIR)

angles = [5, -5, 10, -10, 15, -15, 20, -20, 25, -25, 30, -30, 45, -45, 60, -60,
          90, -90, 120, -120, 150, -150, 180]


# Higher weight for smaller angles
weights = [
    9, 9,       # ±5
    9, 9,       # ±10
    9, 9,       # ±15
    9, 9,       # ±20
    9, 9,       # ±25
    5, 5,       # ±30
    5, 5,       # ±45
    5, 5,       # ±60
    3, 3,       # ±90
    3, 3,       # ±120
    1, 1,       # ±150
    1           # 180
]


def rotate_image(img, angle):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    mat = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Compute rotated bounding box to avoid cropping
    cos = np.abs(mat[0, 0])
    sin = np.abs(mat[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust for translation
    mat[0, 2] += (new_w / 2) - center[0]
    mat[1, 2] += (new_h / 2) - center[1]

    rotated = cv2.warpAffine(img, mat, (new_w, new_h), flags=cv2.INTER_LINEAR)
    return rotated

def zoom_image(img, zoom_factor):
    """Zoom in or zoom out while keeping final size same."""
    h, w = img.shape[:2]

    # Zoom in (crop + resize back)
    if zoom_factor > 1:
        new_h = int(h / zoom_factor)
        new_w = int(w / zoom_factor)
        y1 = (h - new_h) // 2
        x1 = (w - new_w) // 2
        cropped = img[y1:y1+new_h, x1:x1+new_w]
        return cv2.resize(cropped, (w, h))

    # Zoom out (pad + resize back)
    else:
        small = cv2.resize(img, (int(w * zoom_factor), int(h * zoom_factor)))
        new_img = np.zeros_like(img)
        y_start = (h - small.shape[0]) // 2
        x_start = (w - small.shape[1]) // 2
        new_img[y_start:y_start+small.shape[0], x_start:x_start+small.shape[1]] = small
        return new_img

def augment_image(img_path, save_dir):
    img = cv2.imread(img_path)
    base_name = os.path.splitext(os.path.basename(img_path))[0]

    # 1) Redundant original copies
    for i in range(1, 2):
        cv2.imwrite(os.path.join(OUTPUT_DIR,f"{base_name}_orig{i}.png"), img)

    # 2) Zoom in / out
    zoom_in_factor = random.uniform(1,1.25)
    zoom_out_factor = random.uniform(0.75,1)
    zoom_in = zoom_image(img, zoom_in_factor)
    zoom_out = zoom_image(img, zoom_out_factor)

    cv2.imwrite(os.path.join(OUTPUT_DIR,f"{base_name}_zoom_in.png"), zoom_in)
    cv2.imwrite(os.path.join(OUTPUT_DIR, f"{base_name}_zoom_out.png"), zoom_out)

    # 3) 3 rotations w/ weighted probability
    for i in range(1, 3):
        angle = random.choices(angles, weights=weights, k=1)[0]
        rot_angle = angle + random.uniform(-1.5,1.5)
        rotated = rotate_image(img, rot_angle)
        cv2.imwrite(os.path.join(OUTPUT_DIR, f"{base_name}_rot_{angle}_{i}_soft.png"), rotated)
    for i in range(1, 5):
        angle = random.choices(angles, weights=weights, k=1)[0]
        rot_angle = angle
        rotated = rotate_image(img, rot_angle)
        cv2.imwrite(os.path.join(OUTPUT_DIR, f"{base_name}_rot_{angle}_{i}_hard.png"), rotated)


# Example usage
files = os.listdir(DATASET_DIR)
for image_path in files:
    if not image_path.endswith((".png", ".jpg", ".jpeg")):
        continue
    augment_image(os.path.join(DATASET_DIR,image_path), OUTPUT_DIR)

# from multiprocessing import Pool, cpu_count
# import os

# if __name__ == "__main__":
#     image_paths = [os.path.join(DATASET_DIR, f) for f in os.listdir(DATASET_DIR)]

#     with Pool(cpu_count()) as p:
#         p.starmap(augment_image, [(img_path, OUTPUT_DIR) for img_path in image_paths])

#     print("Augmentation completed!")

print("Augmentation completed!")


Augmentation completed!


In [14]:
len(os.listdir(OUTPUT_DIR))

1065

In [8]:
import shutil
count = 0
DATASET_DIR = "/content/drive/MyDrive/Dataset/ECG/"
for filename in os.listdir(DATASET_DIR):
    src_path = os.path.join(DATASET_DIR, filename)
    dst_path = os.path.join(OUTPUT_DIR, filename)

    if os.path.isfile(src_path):
        shutil.copy(src_path, dst_path)
        count += 1

print(f"Copied {count} files from augmented → input folder!")

Copied 93 files from augmented → input folder!


In [15]:
import os
import shutil
import random

DATASET_DIR = "/content/drive/MyDrive/Dataset/ECG/augmented/"
VAL_DIR = "/content/drive/MyDrive/Dataset/ECG_val/"

# Create validation directory if not exists
os.makedirs(VAL_DIR, exist_ok=True)

# List all image files
all_files = [
    f for f in os.listdir(DATASET_DIR)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
]

print(f"Total images found: {len(all_files)}")

# Shuffle deterministically for reproducibility
random.seed(42)
random.shuffle(all_files)

# Compute number of validation samples (10%)
val_count = int(0.10 * len(all_files))

val_files = all_files[:val_count]  # first 10% after shuffle

print(f"Moving {val_count} images to validation folder...")

moved = 0
for filename in val_files:
    src_path = os.path.join(DATASET_DIR, filename)
    dst_path = os.path.join(VAL_DIR, filename)

    if os.path.isfile(src_path):
        shutil.move(src_path, dst_path)
        moved += 1

print("Done!")
print(f"Validation images moved: {moved}")
print(f"Training images remaining: {len(all_files) - moved}")


Total images found: 1065
Moving 106 images to validation folder...
Done!
Validation images moved: 106
Training images remaining: 959


In [16]:
!rm -rf /content/local_data/
!rm -rf /content/local_val/

In [17]:
!cp -r "/content/drive/MyDrive/Dataset/ECG/augmented" "/content/local_data/"

In [18]:
!cp -r "/content/drive/MyDrive/Dataset/ECG_val" "/content/local_val/"

In [11]:
import os
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

DATASET_DIR = "/content/local_data/"
VAL_DIR = "/content/local_val/"

ANGLE_CLASSES = [0, 5, -5, 10, -10, 15, -15, 20, -20, 25, -25, 30, -30, 45, -45, 60, -60,
                90, -90, 120, -120, 150, -150, 180]

angle_to_idx = {a: i for i, a in enumerate(ANGLE_CLASSES)}

IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 25
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



class AngleDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.files = [f for f in os.listdir(folder) if f.lower().endswith((".jpg",".png",".jpeg"))]
        self.transform = transform

    def extract_angle(self, filename):
        """
        Extract rotation angle from filename.
        Valid cases:
          - *_rot_-45_3.png → angle = -45
          - *_rot_30.png    → angle = 30
          - *_orig2.png     → angle = 0
          - *_zoom_in.png   → angle = 0
          - *_zoom_out.png  → angle = 0
          - *.png (no tag)  → angle = 0
        """

        # 1) Check for rotation pattern: "_rot_<angle>"
        rot_match = re.search(r"_rot_(-?\d+)", filename)
        if rot_match:
            angle = int(rot_match.group(1))
            return angle_to_idx[angle]

        # 2) Any augmentation that does not specify angle → assume 0 (label index for zero)
        zero_angle = 0
        return angle_to_idx[zero_angle]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        fname = self.files[idx]
        path = os.path.join(self.folder, fname)

        img = Image.open(path).convert("RGB")
        y = self.extract_angle(fname)

        if self.transform:
            img = self.transform(img)

        return img, y

# imageNet normalization

train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomApply([
        transforms.ColorJitter(0.1,0.1,0.1,0.05)
    ], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])



train_ds = AngleDataset(DATASET_DIR, transform=train_tf)
val_ds   = AngleDataset(VAL_DIR, transform=val_tf)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)



model = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(ANGLE_CLASSES))
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

scaler = torch.cuda.amp.GradScaler()


def evaluate():
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for x, y in val_dl:
            x, y = x.to(DEVICE), y.to(DEVICE)
            preds = model(x)
            correct += (preds.argmax(1) == y).sum().item()
            total += y.size(0)
    return correct / total

best_acc = 0

for epoch in range(EPOCHS):
    model.train()
    loop = tqdm(train_dl, desc=f"Epoch {epoch+1}/{EPOCHS}")

    for imgs, labels in loop:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()

        # Auto Mixed Precision
        with torch.cuda.amp.autocast():
            preds = model(imgs)
            loss = criterion(preds, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss=loss.item())

    acc = evaluate()
    scheduler.step()

    print(f"Val Accuracy: {acc:.4f}")

    # Save best model
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), f"best_mobilenetv3_angle_{acc}.pth")
        print("🔥 Saved new best model!")

print("Training complete!")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth


100%|██████████| 9.83M/9.83M [00:00<00:00, 92.5MB/s]
/tmp/ipython-input-1933980987.py:104: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
/usr/local/lib/python3.12/dist-packages/torch/amp/grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
Epoch 1/25:   0%|          | 0/11 [00:00<?, ?it/s]/tmp/ipython-input-1933980987.py:130: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 1/25: 100%|██████████| 11/11 [04:23<00:00, 23.94s/it, loss=0.992]


Val Accuracy: 0.4422
🔥 Saved new best model!


Epoch 2/25: 100%|██████████| 11/11 [04:29<00:00, 24.47s/it, loss=0.321]


Val Accuracy: 0.5737
🔥 Saved new best model!


...

Val Accuracy: 0.9761


Epoch 18/25: 100%|██████████| 11/11 [04:29<00:00, 24.48s/it, loss=0.00298]


Val Accuracy: 0.9761


Epoch 19/25:  55%|█████▍    | 6/11 [02:48<02:20, 28.10s/it, loss=0.00248]


KeyboardInterrupt: 

In [13]:
files = os.listdir(DATASET_DIR)

angles = [5, -5, 10, -10, 15, -15, 20, -20, 25, -25, 30, -30, 45, -45, 60, -60,
          90, -90, 120, -120, 150, -150, 180]


DATASET_DIR = "/content/drive/MyDrive/Dataset/ECG/"  # change this to your folder
OUTPUT_DIR = "/content/drive/MyDrive/Dataset/ECG/augmented"

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Input folder:", DATASET_DIR)
print("Output folder:", OUTPUT_DIR)

# Higher weight for smaller angles
weights = [
    9, 9,       # ±5
    9, 9,       # ±10
    9, 9,       # ±15
    9, 9,       # ±20
    9, 9,       # ±25
    5, 5,       # ±30
    5, 5,       # ±45
    5, 5,       # ±60
    3, 3,       # ±90
    3, 3,       # ±120
    1, 1,       # ±150
    1           # 180
]


def rotate_image(img, angle):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    mat = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Compute rotated bounding box to avoid cropping
    cos = np.abs(mat[0, 0])
    sin = np.abs(mat[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Adjust for translation
    mat[0, 2] += (new_w / 2) - center[0]
    mat[1, 2] += (new_h / 2) - center[1]

    rotated = cv2.warpAffine(img, mat, (new_w, new_h), flags=cv2.INTER_LINEAR)
    return rotated


def augment_image(img_path, save_dir):
    img = cv2.imread(img_path)
    base_name = os.path.splitext(os.path.basename(img_path))[0]


    # 3) 3 rotations w/ weighted probability
    for i in range(3, 5):
        angle = random.choices(angles, weights=weights, k=1)[0]
        rot_angle = angle + random.uniform(-1.5,1.5)
        rotated = rotate_image(img, rot_angle)
        cv2.imwrite(os.path.join(OUTPUT_DIR, f"{base_name}_rot_{angle}_{i}_soft.png"), rotated)
    for i in range(3, 5):
        angle = random.choices(angles, weights=weights, k=1)[0]
        rot_angle = angle
        rotated = rotate_image(img, rot_angle)
        cv2.imwrite(os.path.join(OUTPUT_DIR, f"{base_name}_rot_{angle}_{i}_hard.png"), rotated)


# Example usage
files = os.listdir(DATASET_DIR)
for image_path in files:
    if not image_path.endswith((".png", ".jpg", ".jpeg")):
        continue
    augment_image(os.path.join(DATASET_DIR,image_path), OUTPUT_DIR)

# from multiprocessing import Pool, cpu_count
# import os

# if __name__ == "__main__":
#     image_paths = [os.path.join(DATASET_DIR, f) for f in os.listdir(DATASET_DIR)]

#     with Pool(cpu_count()) as p:
#         p.starmap(augment_image, [(img_path, OUTPUT_DIR) for img_path in image_paths])

#     print("Augmentation completed!")

print("Augmentation completed!")


Input folder: /content/drive/MyDrive/Dataset/ECG/
Output folder: /content/drive/MyDrive/Dataset/ECG/augmented
Augmentation completed!


In [ ]:
checkpoint_path = "/content/best_mobilenetv3_angle_0.9760956175298805.pth"  # example name

model = models.mobilenet_v3_small(weights="IMAGENET1K_V1")
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(ANGLE_CLASSES))
model = model.to(DEVICE)

# 🔥 load weights
model.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE))

print("Checkpoint loaded! Continuing training...")


In [19]:
import os
import re
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

DATASET_DIR = "/content/local_data/"
VAL_DIR = "/content/local_val/"

ANGLE_CLASSES = [0, 5, -5, 10, -10, 15, -15, 20, -20, 25, -25, 30, -30, 45, -45, 60, -60,
                90, -90, 120, -120, 150, -150, 180]

angle_to_idx = {a: i for i, a in enumerate(ANGLE_CLASSES)}




train_ds = AngleDataset(DATASET_DIR, transform=train_tf)
val_ds   = AngleDataset(VAL_DIR, transform=val_tf)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)



In [20]:
EXTRA_EPOCHS = 10

for epoch in range(10):   # continue numbering
    model.train()
    loop = tqdm(train_dl, desc=f"Continued Epoch {epoch+1}")

    for imgs, labels in loop:
        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        with torch.cuda.amp.autocast():
            preds = model(imgs)
            loss = criterion(preds, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loop.set_postfix(loss=loss.item())

    acc = evaluate()
    scheduler.step()

    print(f"Val Accuracy: {acc:.4f}")

    # Save new best
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), f"best_mobilenetv3_angle_{acc}.pth")
        print("🔥 Saved improved model!")


Continued Epoch 1:   0%|          | 0/15 [00:00<?, ?it/s]/tmp/ipython-input-490428507.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Continued Epoch 1: 100%|██████████| 15/15 [07:07<00:00, 28.52s/it, loss=0.0798]


Val Accuracy: 0.9492


Continued Epoch 2: 100%|██████████| 15/15 [06:47<00:00, 27.15s/it, loss=0.0315]


Val Accuracy: 0.9492


Continued Epoch 3: 100%|██████████| 15/15 [07:11<00:00, 28.78s/it, loss=0.0516]


...

Val Accuracy: 0.9859


Continued Epoch 9: 100%|██████████| 15/15 [06:27<00:00, 25.85s/it, loss=0.0456]


Val Accuracy: 0.9915
🔥 Saved improved model!


Continued Epoch 10: 100%|██████████| 15/15 [06:30<00:00, 26.06s/it, loss=0.00666]


Val Accuracy: 0.9944
🔥 Saved improved model!


In [ ]:
def get_misclassified_images(model, dataset):
    """
    Returns a list of full file paths for misclassified images in the dataset.
    """
    model.eval()
    wrong_paths = []

    with torch.no_grad():
        for idx in range(len(dataset)):
            img, label = dataset[idx]
            path = os.path.join(dataset.folder, dataset.files[idx])

            img_input = img.unsqueeze(0).to(DEVICE)

            pred = model(img_input).argmax(1).item()

            if pred != label:
                wrong_paths.append(path)
                print(pred,label)

    return wrong_paths

wrong_samples = get_misclassified_images(model, val_ds)
print(f"Total misclassified: {len(wrong_samples)}")

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 8))

for p in wrong_samples:
    image = Image.open(p).convert("RGB")
    plt.subplot(1, len(wrong_samples), wrong_samples.index(p) + 1)
    plt.imshow(image)

plt.axis('off')
plt.show()


6 4
0 23


In [22]:
wrong_samples

['/content/local_val/1277179302-0006_rot_-10_3_hard.png',
 '/content/local_val/121471389-0003_rot_180_4_soft.png']

In [ ]:
ANGLE_CLASSES[6], ANGLE_CLASSES[0]